In [10]:
import get_api_data,git_local
import pandas as pd
from flatten_dict import flatten
from datetime import datetime
from dateutil import relativedelta
import os

In [2]:
access_token = 'ef66b5bf5fd6c14d63586bf00caa9e72c78cc675'
repo_owner = 'rspec'
git_url = 'git://github.com/rspec/rspec-rails.git'
api_base_url = 'https://api.github.com'
repo_name = 'rspec-rails'
client = get_api_data.get_data(access_token,repo_owner,api_base_url,repo_name) 

In [3]:
local = git_local.git_local(git_url,repo_name)
local.clone_repo()

pygit2.Repository('/Users/suvodeepmajumder/Documents/Class/CSC510/csc510-project/code_base/git_access/temp_repo/rspec-rails/.git/')

In [4]:
commits = local.get_commits()
commit_df = pd.DataFrame(commits, columns = ['commit_id','commit_time','author','author_email',
                                             'committer','committer_email','comments','commit_parent'])

In [5]:
committed_files = local.get_committed_files()
committed_files_df = pd.DataFrame(committed_files, columns = ['commit_id','file_id','modification_type','file_name'])

In [6]:
diffs = local.get_diffs(commit_df.commit_id.values.tolist())

In [7]:
temp_df = committed_files_df
temp_df = temp_df.drop(labels = ['file_id','modification_type'],axis = 1)
print(temp_df.columns)

Index(['commit_id', 'file_name'], dtype='object')


In [13]:
uc2 = {}
total_file_changed = {}
for commit in temp_df.commit_id.unique():
    files = temp_df[temp_df['commit_id'] == commit].file_name.values.tolist()
    for file_s in files:
        _file = os.getcwd() + '/temp_repo/' + repo_name + '/' + file_s
        exists = os.path.isfile(_file)
        if exists:
            if file_s not in uc2:
                uc2[file_s] = {}
                total_file_changed[file_s] = 1
            else:
                total_file_changed[file_s] += 1
            for file_d in files:
                if file_s == file_d:
                    continue
                else:
                    if file_d not in uc2[file_s]:
                        uc2[file_s][file_d] = 1
                    else:
                        uc2[file_s][file_d] += 1
        else:
            continue
    
    

In [14]:
uc2_matrix = []
for file_s in uc2:
    for file_d in uc2[file_s]:
        uc2_matrix.append([file_s,file_d,uc2[file_s][file_d],uc2[file_s][file_d]/total_file_changed[file_s]])

In [15]:
uc2_matrix_df = pd.DataFrame(uc2_matrix, columns = 
                             ['SourceFilePath','DestinationFilePath','NoOfTimeChanged','NormalizedChange'])

In [16]:
uc2_matrix_df['RepoName'] = [0]*uc2_matrix_df.shape[0]
uc2_matrix_df['RepoUrl'] = [0]*uc2_matrix_df.shape[0]
for i in range(uc2_matrix_df.shape[0]):
    uc2_matrix_df.iloc[i,4] = repo_name
    uc2_matrix_df.iloc[i,5] = git_url

In [17]:
uc2_matrix_df.to_csv('uc2.csv')

In [18]:
uc2_matrix_df

,SourceFilePath,DestinationFilePath,NoOfTimeChanged,NormalizedChange,RepoName,RepoUrl
0,lib/rspec/rails/example/controller_example_gro...,lib/rspec/rails/example/request_example_group.rb,48,0.342857,rspec-rails,git://github.com/rspec/rspec-rails.git
1,lib/rspec/rails/example/controller_example_gro...,example_app_template.rb,5,0.035714,rspec-rails,git://github.com/rspec/rspec-rails.git
2,lib/rspec/rails/example/controller_example_gro...,lib/generators/rspec-rails.rb,1,0.007143,rspec-rails,git://github.com/rspec/rspec-rails.git
3,lib/rspec/rails/example/controller_example_gro...,lib/generators/rspec-rails/controller/controll...,1,0.007143,rspec-rails,git://github.com/rspec/rspec-rails.git
4,lib/rspec/rails/example/controller_example_gro...,lib/generators/rspec-rails/controller/template...,1,0.007143,rspec-rails,git://github.com/rspec/rspec-rails.git
5,lib/rspec/rails/example/controller_example_gro...,lib/generators/rspec-rails/controller/template...,1,0.007143,rspec-rails,git://github.com/rspec/rspec-rails.git
6,lib/rspec/rails/example/controller_example_gro...,lib/generators/rspec-rails/helper/helper_gener...,1,0.007143,rspec-rails,git://github.com/rspec/rspec-rails.git
7,lib/rspec/rails/example/controller_example_gro...,lib/generators/rspec-rails/helper/templates/he...,1,0.007143,rspec-rails,git://github.com/rspec/rspec-rails.git
8,lib/rspec/rails/example/controller_example_gro...,lib/generators/rspec-rails/install/install_gen...,1,0.007143,rspec-rails,git://github.com/rspec/rspec-rails.git
9,lib/rspec/rails/example/controller_example_gro...,lib/generators/rspec-rails/install/templates/l...,1,0.007143,rspec-rails,git://github.com/rspec/rspec-rails.git


In [8]:
temp_df

,commit_id,file_name
0,15e1656a537bd2b2ce49a294870c68e3707391d6,lib/rspec/rails/example/controller_example_gro...
1,eba85a35ff08cf88bc9e6665c1a46c4b5dc4864f,.gitignore
2,eba85a35ff08cf88bc9e6665c1a46c4b5dc4864f,Rakefile
3,a2e61070b4f432889750b9296d231e39dbe57d2f,lib/rspec/rails/example/request_example_group.rb
4,857a0b3b14e59cb68a530e6a9815d59316f4be6b,lib/rspec/rails/example/controller_example_gro...
5,857a0b3b14e59cb68a530e6a9815d59316f4be6b,lib/rspec/rails/example/request_example_group.rb
6,d85243d328670b70de1f40f4a52a23d60f596e0b,.gitignore
7,c487e51095a66a517ef698c5d23fa04a7f042c92,Rakefile
8,c487e51095a66a517ef698c5d23fa04a7f042c92,example_app_template.rb
9,f06aa507280f8f9e6316c9bd5f72e640269a49b1,example_app_template.rb
